In [8]:
# useing this variable for saving models and summaries
exp_name = "RCNN_Wmultitask_16_opp3"

## Importing Libraries

In [9]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../../src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import models.CNN_lstm_model as CNN_lstm_model
import training
import evaluation.nn_eval_multitask as nn_eval

## Loading synthetic data

In [11]:
#loading data
data_path= '../../data/opp8.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)

In [12]:
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

In [13]:
train_data.shape, train_gt.shape

((691, 1024, 77), (691, 1024, 19))

In [14]:
test_data.shape, test_gt.shape

((119, 1024, 77), (119, 1024, 19))

## Model

In [15]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 100

pooling_stride = [4, 2, 2]
weights_shape = [[5, n_variables, 32],
                 [3, 32, 32],
                 [3, 32, 32]]
bias_shape = [32, 32, 32]
activation = tf.nn.relu

lstm_n_hidden = 32
n_classes = 19

##positive weight for weighted cross entropy
s = train_gt.shape
resolution = np.prod(pooling_stride)

pooled_gt = np.reshape(train_gt[:,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)

variables_weights = np.ones((n_classes,))

In [16]:
pos_weight.shape

(19,)

In [17]:
pos_weight

array([  0.36565482,  48.46756152,  47.65126513,  50.0669746 ,
        49.94930876,  40.02411874,  41.52307692,  58.84303112,
        63.4664723 ,  75.37996546,  86.74603175,  80.14495413,
        87.27145709,  69.42038217,  68.86413902,  43.71587462,
        13.05721551,  58.2021419 ,  10.05323669])

In [18]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [19]:
summary_path = '../../summary/' + exp_name 
checkpoint_path = '../../model/' + exp_name + '/' + exp_name 

In [20]:
model = CNN_lstm_model.Model(n_variables = n_variables,
                        n_classes = n_classes)

In [21]:
model.build_cnn_lstm_net(pooling_stride = pooling_stride,
                         weights_shape = weights_shape,
                         bias_shape = bias_shape,
                         activation = activation,
                         lstm_n_hidden = lstm_n_hidden,
                         n_classes = n_classes,
                         learning_rate = learning_rate,
                         pos_weight = pos_weight,
                         trainable_conv = True,
                         trainable_last = True,
                         variables_weights = variables_weights)

In [22]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0,
                    eval_type = 'multitask')

In [23]:
trainer.train(max_iter = 10000,
                train_eval_step = 100, 
                validation_eval_step = 100,
                display_step = 100)

Training: 0
ACTIVITY RECOGNITION
AUC = 0.069 vs. 0.061
F1 = 0.099 vs. 0.091
Accuracy = 0.013 vs. 0.056
CHANGE DETECTION
AUC = 0.096 vs. 0.086
F1 = 0.197 vs. 0.165
Precision = 0.113 vs. 0.090
Recall = 0.766 vs. 0.998
Validation: 0
ACTIVITY RECOGNITION
AUC = 0.062 vs. 0.058
F1 = 0.083 vs. 0.080
Accuracy = 0.002 vs. 0.056
CHANGE DETECTION
AUC = 0.062 vs. 0.056
F1 = 0.144 vs. 0.101
Precision = 0.079 vs. 0.054
Recall = 0.805 vs. 0.717
Minibatch Loss= 1.738655
Training: 100
ACTIVITY RECOGNITION
AUC = 0.488 vs. 0.061
F1 = 0.534 vs. 0.094
Accuracy = 0.489 vs. 0.056
CHANGE DETECTION
AUC = 0.229 vs. 0.102
F1 = 0.323 vs. 0.188
Precision = 0.206 vs. 0.104
Recall = 0.751 vs. 0.956
Validation: 100
ACTIVITY RECOGNITION
AUC = 0.291 vs. 0.058
F1 = 0.363 vs. 0.081
Accuracy = 0.418 vs. 0.056
CHANGE DETECTION
AUC = 0.164 vs. 0.047
F1 = 0.259 vs. 0.096
Precision = 0.168 vs. 0.051
Recall = 0.566 vs. 0.990
Minibatch Loss= 0.519431
Training: 200
ACTIVITY RECOGNITION
AUC = 0.617 vs. 0.062
F1 = 0.635 vs. 0.091


In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [18]:
#load the model
#saver = tf.train.Saver()
#saver.restore(sess, "../model" + "/CNN7002_Gleam" + "/CNN7002_Gleam")
#saver.restore(sess, '../model/CNN7002_Gleam/CNN7002_Gleam-10000')


In [25]:
#This is the model evaluation on test dataset
_= nn_eval.evaluate_model (model, sess, test_data, test_gt)

ACTIVITY RECOGNITION
AUC = 0.599 vs. 0.060
F1 = 0.608 vs. 0.084
Accuracy = 0.858 vs. 0.056
CHANGE DETECTION
AUC = 0.291 vs. 0.049
F1 = 0.343 vs. 0.097
Precision = 0.248 vs. 0.051
Recall = 0.556 vs. 0.935


In [26]:
out = nn_eval.deploy (model, sess, test_data, test_gt)

In [27]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
#probability_change = np.squeeze(probability_of_pos, axis = 2)

In [28]:
savemat('../../poc/poc'+exp_name+'.mat', {'poc':out[:,:,-1]})

In [25]:
1

1